# SQL in Python (Exercises)

In [40]:
# Imports

import sqlite3
import pandas as pd

In [41]:
# Load data

data = pd.read_csv('https://raw.githubusercontent.com/renatomaaliw3/public_files/refs/heads/master/Data%20Sets/nba%20cleaned.csv')
data.head(5)

,Name,Team,Position,Height,Weight,College,Salary
0,Saddiq Bey,Atlanta Hawks,F,6-7,215,Villanova,4556983
1,Bogdan Bogdanovic,Atlanta Hawks,G,6-5,225,Fenerbahce,18700000
2,Kobe Bufkin,Atlanta Hawks,G,6-5,195,Michigan,4094244
3,Clint Capela,Atlanta Hawks,C,6-10,256,Elan Chalon,20616000
4,Bruno Fernando,Atlanta Hawks,F-C,6-10,240,Maryland,2581522


In [42]:
# Modifications

data['Height_'] = data['Height'].str.replace('-', '.').astype(float)

In [43]:
# Connect to an in memory SQLite Database

conn = sqlite3.connect(':memory:')

In [44]:
# Write the DataFrame to the Database

data.to_sql('NBA', conn, index = False, if_exists = 'replace')

583

In [45]:
# Execute a SQL Query

query = "SELECT * FROM NBA"
result_sql = pd.read_sql_query(query, conn)
result_sql

,Name,Team,Position,Height,Weight,College,Salary,Height_
0,Saddiq Bey,Atlanta Hawks,F,6-7,215,Villanova,4556983,6.7
1,Bogdan Bogdanovic,Atlanta Hawks,G,6-5,225,Fenerbahce,18700000,6.5
2,Kobe Bufkin,Atlanta Hawks,G,6-5,195,Michigan,4094244,6.5
3,Clint Capela,Atlanta Hawks,C,6-10,256,Elan Chalon,20616000,6.1
4,Bruno Fernando,Atlanta Hawks,F-C,6-10,240,Maryland,2581522,6.1
...,...,...,...,...,...,...,...,...
578,Jordan Poole,Washington Wizards,G,6-4,194,Michigan,27955357,6.4
579,Ryan Rollins,Washington Wizards,G,6-3,180,Toledo,1719864,6.3
580,Landry Shamet,Washington Wizards,G,6-4,190,Wichita State,10250000,6.4
581,Tristan Vukcevic,Washington Wizards,F,6-10,220,Real Madrid,0,6.1


In [46]:
# 01. What is the total payroll for the Golden State Warriors

query = "SELECT SUM(Salary) AS [Total Salary] FROM NBA WHERE Team = 'Golden State Warriors'"
result_sql = pd.read_sql_query(query, conn)
result_sql

,Total Salary
0,209671207


In [47]:
# 02. Can you identify what team is the tallest in terms of average player height?

query = "SELECT Team, AVG(CAST(REPLACE(Height, '-', '.') AS REAL)) AS 'AVG_Height' "
query += "FROM NBA GROUP BY Team ORDER BY AVG_Height DESC"

result_sql = pd.read_sql_query(query, conn)
result_sql

,Team,AVG_Height
0,Sacramento Kings,6.677778
1,Portland Trail Blazers,6.658824
2,Boston Celtics,6.650000
3,Toronto Raptors,6.635000
4,Charlotte Hornets,6.606000
5,Los Angeles Clippers,6.600000
6,Los Angeles Lakers,6.590000
7,New York Knicks,6.580952
8,Indiana Pacers,6.571905
9,Oklahoma City Thunder,6.571905


In [63]:
# 03. Who are the tallest and shortest player in the NBA? (Hint: We will use UNION to display in one query)

query = "SELECT * FROM NBA WHERE Height_ = (SELECT MAX(Height_) FROM NBA WHERE Salary != 0) "
query += "UNION "
query += "SELECT * FROM NBA WHERE Height_ = (SELECT MIN(Height_) FROM NBA WHERE Salary != 0) "

result_sql = pd.read_sql_query(query, conn)
result_sql

,Name,Team,Position,Height,Weight,College,Salary,Height_
0,Markquis Nowell,Toronto Raptors,G,5-8,160,Kansas State,559782,5.8
1,Victor Wembanyama,San Antonio Spurs,F-C,7-4,210,Metropolitans 92,12160632,7.4


In [49]:
# 04. Which team has the most players who play two positions?

query = "SELECT Team, COUNT(Position) AS [2WayPlayers] FROM NBA "
query += "WHERE Position LIKE '%-%' "
query += "GROUP BY Team "
query += "ORDER BY [2WayPlayers] DESC Limit 5"

result_sql = pd.read_sql_query(query, conn)
result_sql

,Team,2WayPlayers
0,New York Knicks,8
1,Indiana Pacers,8
2,Boston Celtics,8
3,San Antonio Spurs,7
4,Phoenix Suns,7


In [50]:
# 05. Find the average salary of players grouped by each unique height,
#     and filter only the heights with more than 3 players.
#     Then, display the results sorted by average salary in descending order.

query = "SELECT Height, COUNT(*) AS [Player_Count], AVG(Salary) AS [Average_Salary] FROM NBA "
query += "GROUP BY Height "
query += "HAVING COUNT(*) > 3 "
query += "ORDER BY Average_Salary DESC "

result_sql = pd.read_sql_query(query, conn)
result_sql

,Height,Player_Count,Average_Salary
0,7-1,33,1.233314e+07
1,6-2,29,1.053380e+07
2,6-1,31,1.004137e+07
3,6-4,53,8.234721e+06
4,6-8,60,8.113891e+06
5,6-10,36,8.023816e+06
6,6-5,74,7.426138e+06
7,6-7,72,6.909398e+06
8,6-6,69,6.877540e+06
9,6-11,30,6.306839e+06


In [51]:
#06. Find the highest-paid/lowest salary within each team,
#    along with the player’s name and salary.

query = "SELECT Name, Team, Position, MIN(Salary) AS [Highest_Salary] FROM NBA "
query += "GROUP BY Team "
query += "ORDER BY Highest_Salary DESC "

result_sql = pd.read_sql_query(query, conn)
result_sql

,Name,Team,Position,Highest_Salary
0,Joey Hauser,Utah Jazz,F,559782
1,Keyontae Johnson,Oklahoma City Thunder,F,559782
2,Trent Forrest,Atlanta Hawks,G,508891
3,Taj Gibson,Washington Wizards,F,0
4,O.G. Anunoby,Toronto Raptors,F,0
5,Devonte' Graham,San Antonio Spurs,G,0
6,Aleksandar Vezenkov,Sacramento Kings,F,0
7,John Butler Jr.,Portland Trail Blazers,F,0
8,Bismack Biyombo,Phoenix Suns,C,0
9,Ricky Council IV,Philadelphia 76ers,G,0


In [52]:
#07. Calculate the salary disparity within each team by finding the difference between
#    the highest-paid and lowest-paid player on each team.

query = "SELECT Team, MAX(Salary) - MIN(Salary) AS [Salary_Disparity] FROM NBA "
query += "GROUP BY Team "
query += "ORDER BY Salary_Disparity DESC "

result_sql = pd.read_sql_query(query, conn)
result_sql

,Team,Salary_Disparity
0,Golden State Warriors,51915615
1,Phoenix Suns,47649433
2,Philadelphia 76ers,47607350
3,Los Angeles Lakers,47607350
4,Denver Nuggets,47607350
5,Portland Trail Blazers,45640084
6,Milwaukee Bucks,45640084
7,Los Angeles Clippers,45640084
8,Miami Heat,45183960
9,Minnesota Timberwolves,41000000


In [53]:
#08. Find the total number of players league-wide who can
#    play multiple positions of (F-G) or (G-F) and display this count.


query = "SELECT COUNT(*) AS [Multi_Position] FROM NBA "
query += "WHERE Position IN ('F-G', 'G-F') "

result_sql = pd.read_sql_query(query, conn)
result_sql

,Multi_Position
0,61


In [55]:
#09. Which team has the most players that belongs to the same college?

query = "SELECT Team, College, COUNT(*) AS [Same_College] FROM NBA "
query += "WHERE COLLEGE IS NOT NULL "
query += "GROUP BY Team, College "
query += "ORDER BY Same_College DESC LIMIT 5"

result_sql = pd.read_sql_query(query, conn)
result_sql

,Team,College,Same_College
0,Orlando Magic,Michigan,4
1,Sacramento Kings,Kentucky,4
2,Brooklyn Nets,Louisiana State,3
3,Dallas Mavericks,Duke,3
4,Indiana Pacers,Arizona,3


In [58]:
#10. Find the highest salary for each unique positions in the league, display the name and team of the player

query = "SELECT Name, Team, Position, MAX(Salary) AS [Highest_Salary_Per_Position] FROM NBA "
query += "GROUP BY Position "
query += "ORDER BY Highest_Salary_Per_Position DESC"

result_sql = pd.read_sql_query(query, conn)
result_sql

,Name,Team,Position,Highest_Salary_Per_Position
0,Stephen Curry,Golden State Warriors,G,51915615
1,Kevin Durant,Phoenix Suns,F,47649433
2,Joel Embiid,Philadelphia 76ers,C-F,47607350
3,Nikola Jokic,Denver Nuggets,C,47607350
4,Anthony Davis,Los Angeles Lakers,F-C,40600080
5,Luka Doncic,Dallas Mavericks,F-G,40064220
6,Ben Simmons,Brooklyn Nets,G-F,37893408
